## Creating a Spark Session

Using Kafka consumer and producer, a data pipeline was created that reads live data from the National Rail Data Portal and sends messages for each event into our PostgreSQL database, hosted on Amazon Web Services.

The following data and alaysis is intented for scheduling prediction purposes. This report details data cleaning and analysis, focusing on...

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DecimalType
import os

# Creating a SparkSession

spark = SparkSession \
    .builder \
    .appName("MySparkApp") \
    .config("spark.jars", "postgresql-42.7.3.jar") \
    .getOrCreate()

## Reading in data from PostgreSQL

In [5]:
properties = {
    'HOSTNAME' : 'database-3.cfiq0ygikewk.us-east-2.rds.amazonaws.com',
    'PORT' : 5432,
    'DBNAME' : 'postgres',
    'USERNAME' : 'root',
    'PASS' : 'brianna123'
}

df = spark.read \
    .format("jdbc") \
    .option("url", f"jdbc:postgresql://{properties['HOSTNAME']}:{properties['PORT']}/{properties['DBNAME']}") \
    .option("dbtable", "darwin") \
    .option("user", properties['USERNAME']) \
    .option("password", properties['PASS']) \
    .option("driver", "org.postgresql.Driver") \
    .load()

df.printSchema()

root
 |-- route_id: string (nullable = true)
 |-- unique_id: string (nullable = true)
 |-- service_start_date: date (nullable = true)
 |-- update_origin: string (nullable = true)
 |-- train_platform: string (nullable = true)
 |-- working_time_pass: timestamp (nullable = true)
 |-- working_time_arrival: timestamp (nullable = true)
 |-- working_time_departure: timestamp (nullable = true)
 |-- planned_time_arrival: timestamp (nullable = true)
 |-- planned_time_departure: timestamp (nullable = true)
 |-- actual_arrival_time: timestamp (nullable = true)
 |-- actual_departure_time: timestamp (nullable = true)
 |-- platform: string (nullable = true)
 |-- train_length: integer (nullable = true)
 |-- estimated_time: timestamp (nullable = true)
 |-- source: string (nullable = true)
 |-- actual_time: timestamp (nullable = true)
 |-- actual_time_class: string (nullable = true)
 |-- is_delayed_arrival: boolean (nullable = true)
 |-- is_delayed_departure: boolean (nullable = true)
 |-- source_instan

## Read in Data to Pandas

In [6]:
pandas_df = df.toPandas()
pandas_df

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_pass,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,...,platform,train_length,estimated_time,source,actual_time,actual_time_class,is_delayed_arrival,is_delayed_departure,source_instance,estimated_time_minutes
0,202407087154582,G54582,2024-07-08,TD,CRSHLTN,NaT,1970-01-01 15:48:30,1970-01-01 15:49:30,1970-01-01 15:49:00,1970-01-01 15:49:00,...,1,8.0,NaT,None,NaT,None,False,False,None,NaN
1,202407086735809,C35809,2024-07-08,TD,BRMBRK,NaT,1970-01-01 14:50:00,1970-01-01 14:50:30,1970-01-01 14:50:00,1970-01-01 14:50:00,...,1,NaN,NaT,None,NaT,None,False,False,None,NaN
2,202407086785115,C85115,2024-07-08,TD,SLOUGH,NaT,1970-01-01 14:50:30,1970-01-01 14:51:30,1970-01-01 14:51:00,1970-01-01 14:51:00,...,5,NaN,NaT,None,NaT,None,False,False,None,NaN
3,202407087145471,G45471,2024-07-08,TD,OXFD,NaT,1970-01-01 14:51:30,1970-01-01 14:52:30,1970-01-01 14:52:00,1970-01-01 14:52:00,...,4,NaN,NaT,None,NaT,None,False,False,None,NaN
4,202407087661793,L61793,2024-07-08,TD,BNTFORD,NaT,1970-01-01 14:52:00,1970-01-01 14:53:00,1970-01-01 14:52:00,1970-01-01 14:53:00,...,2,NaN,NaT,None,NaT,None,False,False,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90160,202407156747041,C47041,2024-07-15,CIS,PBRO,NaT,1970-01-01 20:43:00,1970-01-01 20:46:00,1970-01-01 20:43:00,1970-01-01 20:46:00,...,3,NaN,NaT,None,NaT,None,False,False,None,NaN
90161,202407158097374,P97374,2024-07-15,TD,BEXLEY,NaT,1970-01-01 20:46:30,1970-01-01 20:47:30,1970-01-01 20:47:00,1970-01-01 20:47:00,...,2,10.0,NaT,None,NaT,None,False,False,None,NaN
90162,202407157659769,L59769,2024-07-15,TD,WDON,NaT,1970-01-01 20:49:00,1970-01-01 20:50:00,1970-01-01 20:49:00,1970-01-01 20:50:00,...,5,NaN,NaT,None,NaT,None,False,False,None,NaN
90163,202407156779582,C79582,2024-07-15,Trust,SOTON,NaT,1970-01-01 20:43:00,1970-01-01 20:45:00,1970-01-01 20:43:00,1970-01-01 20:45:00,...,3B,NaN,NaT,None,NaT,None,False,False,None,NaN


## Check for Duplicate Values/ Nulls

To ensure quality & accuracy, certain columns containing duplicates or null values will be dropped. 


In [7]:
# Identify duplicate rows
df.groupBy(df.columns)\
    .count()\
    .where(F.col('count') > 1)\
    .select(F.sum('count'))\
    .show()

+----------+
|sum(count)|
+----------+
|      4765|
+----------+



3594 rows containing duplicates will be removed.

In [8]:
# Drop duplicates
df = df.dropDuplicates()

In [9]:
# Null count for all columns
Dict_Null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

{'route_id': 0,
 'unique_id': 0,
 'service_start_date': 0,
 'update_origin': 2623,
 'train_platform': 0,
 'working_time_pass': 87655,
 'working_time_arrival': 0,
 'working_time_departure': 0,
 'planned_time_arrival': 5927,
 'planned_time_departure': 5910,
 'actual_arrival_time': 15689,
 'actual_departure_time': 6299,
 'platform': 4287,
 'train_length': 55847,
 'estimated_time': 87741,
 'source': 87741,
 'actual_time': 87741,
 'actual_time_class': 87741,
 'is_delayed_arrival': 0,
 'is_delayed_departure': 0,
 'source_instance': 87741,
 'estimated_time_minutes': 87741}

In [10]:
# Drop columns with too many nulls

df = df.drop('working_time_pass',\
             'pass',\
             'estimated_time',\
             'actual_time',\
             'actual_time_class',\
             'source_instance',\
             'estimated_time_minutes',\
             'train_length',\
             'source')

In [11]:
# Drop rows with too many nulls
df = df.dropna(subset='planned_time_departure')
df = df.dropna(subset='update_origin')

# Updated column count
df.count()

79215

In [12]:
# Null count for all columns
Dict_Null = {col:df.filter(df[col].isNull()).count() for col in df.columns}
Dict_Null

{'route_id': 0,
 'unique_id': 0,
 'service_start_date': 0,
 'update_origin': 0,
 'train_platform': 0,
 'working_time_arrival': 0,
 'working_time_departure': 0,
 'planned_time_arrival': 133,
 'planned_time_departure': 0,
 'actual_arrival_time': 14880,
 'actual_departure_time': 6007,
 'platform': 1669,
 'is_delayed_arrival': 0,
 'is_delayed_departure': 0}

**Remove outliers**

In [13]:
from pyspark.sql.types import NumericType

def remove_outliers(df, column):
    # Get IQR
    quantiles = df.approxQuantile(column, [0.25, 0.75], 0.05)
    Q1 = quantiles[0]
    Q3 = quantiles[1]
    IQR = Q3 - Q1

    # Define lower and upper bounds, return
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df.filter((F.col(column) >= lower_bound) & (F.col(column) <= upper_bound))

# Get all numeric columns and remove outliers
numeric_columns = [x.name for x in df.schema.fields if isinstance(x.dataType, NumericType)]
for column in numeric_columns:
    df = remove_outliers(df, column)

In [14]:
# Get list of categorical var cols
categorical_columns = [c for c in df.columns if df.select(c).dtypes[0][1] == 'string']

# Value counts for each categorical var
for col_name in categorical_columns:
    print(f"Unique values in {col_name}:")
    df.groupBy(col_name).count().show()

Unique values in route_id:
+---------------+-----+
|       route_id|count|
+---------------+-----+
|202407107150485|   19|
|202407116717283|    1|
|202407116776697|    2|
|202407128082375|   22|
|202407156735818|   10|
|202407087661793|    1|
|202407087657787|   14|
|202407097138619|    6|
|202407107111306|    3|
|202407118095836|    3|
|202407096785250|    7|
|202407107130507|   12|
|202407158097299|    1|
|202407158089453|    1|
|202407128095397|    4|
|202407118081164|    1|
|202407126735520|    9|
|202407098098324|   22|
|202407118090083|    4|
|202407126764994|   12|
+---------------+-----+
only showing top 20 rows

Unique values in unique_id:
+---------+-----+
|unique_id|count|
+---------+-----+
|   C39248|   16|
|   C85163|   17|
|   P95240|   28|
|   C47082|   12|
|   L58897|   16|
|   L60029|    3|
|   G32617|   12|
|   P97113|    6|
|   P96398|   11|
|   P90801|   23|
|   P94454|   10|
|   P93148|   22|
|   C35504|    2|
|   P96483|    4|
|   P85967|   11|
|   P98384|   24|
|

## Merge Data with Rail References

SQL data is merged with RailReferences to idenfity train and station information.

In [15]:
# Import RailReferences.csv & merge with SQL data
rail_ref = spark.read.csv('RailReferences.csv', header=True, inferSchema=True)
rail_ref.printSchema()

root
 |-- AtcoCode: string (nullable = true)
 |-- TiplocCode: string (nullable = true)
 |-- CrsCode: string (nullable = true)
 |-- StationName: string (nullable = true)
 |-- StationNameLang: string (nullable = true)
 |-- GridType: string (nullable = true)
 |-- Easting: integer (nullable = true)
 |-- Northing: integer (nullable = true)
 |-- CreationDateTime: timestamp (nullable = true)
 |-- ModificationDateTime: timestamp (nullable = true)
 |-- RevisionNumber: integer (nullable = true)
 |-- Modification: string (nullable = true)



In [16]:
# Merge both DataFrames for Station Name
merged_df = df.join(rail_ref, df['train_platform'] == rail_ref['TiplocCode'], how='left')
merged_df.count()

79269

## Creating New Columns/ Feature Engineering
New columns:
- planned arrival hour
- planned departure hour
- actual arrival 
- actual depatrue hour
- arrival & depature delays in minutes
- day of week
for further analysis of train delays

In [17]:
# Create separate columns for date, time, hour
from pyspark.sql.functions import *
import datetime

# Planned arrival hour
merged_df = merged_df.withColumn('planned_arrival_hour', hour(F.col('planned_time_arrival')))

# Planned departure hour
merged_df = merged_df.withColumn('planned_dep_hour', hour(F.col('planned_time_departure')))

# Actual arrival hour
merged_df = merged_df.withColumn('actual_arrival_hour', hour(F.col('actual_arrival_time')))

# Actual departure hour
merged_df = merged_df.withColumn('actual_dep_hour', hour(F.col('actual_departure_time')))

# Calculating delay in minutes
merged_df = merged_df.withColumn('arrival_time_delayed_by(min)', (unix_timestamp("actual_arrival_time") - unix_timestamp('planned_time_arrival'))/60)
merged_df = merged_df.withColumn('dep_time_delayed_by(min)', (unix_timestamp("actual_departure_time") - unix_timestamp('planned_time_departure'))/60)

# Create weekday var
merged_df = merged_df.withColumn('day_of_week', date_format(F.col('service_start_date'), 'EEEE'))

# Filter by arrival time delayed & sort
merged_df.select(F.col('arrival_time_delayed_by(min)')).filter(F.col('arrival_time_delayed_by(min)') > 0).orderBy('arrival_time_delayed_by(min)', ascending=False).show()

+----------------------------+
|arrival_time_delayed_by(min)|
+----------------------------+
|                       145.0|
|                       144.0|
|                       141.0|
|                       141.0|
|                       141.0|
|                       140.0|
|                       139.0|
|                       139.0|
|                       139.0|
|                       139.0|
|                       139.0|
|                       139.0|
|                       138.0|
|                       138.0|
|                       137.0|
|                       137.0|
|                       137.0|
|                       137.0|
|                       136.0|
|                       136.0|
+----------------------------+
only showing top 20 rows



In [18]:
#merged_df.show()
merged_df.printSchema()

root
 |-- route_id: string (nullable = true)
 |-- unique_id: string (nullable = true)
 |-- service_start_date: date (nullable = true)
 |-- update_origin: string (nullable = true)
 |-- train_platform: string (nullable = true)
 |-- working_time_arrival: timestamp (nullable = true)
 |-- working_time_departure: timestamp (nullable = true)
 |-- planned_time_arrival: timestamp (nullable = true)
 |-- planned_time_departure: timestamp (nullable = true)
 |-- actual_arrival_time: timestamp (nullable = true)
 |-- actual_departure_time: timestamp (nullable = true)
 |-- platform: string (nullable = true)
 |-- is_delayed_arrival: boolean (nullable = true)
 |-- is_delayed_departure: boolean (nullable = true)
 |-- AtcoCode: string (nullable = true)
 |-- TiplocCode: string (nullable = true)
 |-- CrsCode: string (nullable = true)
 |-- StationName: string (nullable = true)
 |-- StationNameLang: string (nullable = true)
 |-- GridType: string (nullable = true)
 |-- Easting: integer (nullable = true)
 |-- N

## Convert to Pandas for Additional Exploration & Visualization

In [20]:
pandas_df = merged_df.toPandas()

In [22]:
import pandas as pd
# Setting pandas to display all columns 
pd.set_option('display.max_columns', None)

In [23]:
pandas_df

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,is_delayed_arrival,is_delayed_departure,AtcoCode,TiplocCode,CrsCode,StationName,StationNameLang,GridType,Easting,Northing,CreationDateTime,ModificationDateTime,RevisionNumber,Modification,planned_arrival_hour,planned_dep_hour,actual_arrival_hour,actual_dep_hour,arrival_time_delayed_by(min),dep_time_delayed_by(min),day_of_week
0,202407086746668,C46668,2024-07-08,TD,DONC,1970-01-01 14:08:00,1970-01-01 14:10:00,1970-01-01 14:08:00,1970-01-01 14:10:00,NaT,1970-01-01 14:54:00,4,False,False,9100DONC,DONC,DON,Doncaster Rail Station,None,U,457125.0,403175.0,2003-11-04,2010-12-20 15:37:54,2.0,rev,14.0,14,NaN,14.0,NaN,44.0,Monday
1,202407086765055,C65055,2024-07-08,Darwin,PELAWTW,1970-01-01 14:07:00,1970-01-01 14:07:30,1970-01-01 14:07:00,1970-01-01 14:07:00,1970-01-01 15:10:00,1970-01-01 15:10:00,None,True,True,None,None,None,None,None,None,NaN,NaN,NaT,NaT,NaN,None,14.0,14,15.0,15.0,63.0,63.0,Monday
2,202407088094162,P94162,2024-07-08,Trust,CLPHHS,1970-01-01 14:52:30,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:54:00,2,False,False,9100CLPHHS,CLPHHS,CLP,Clapham High Street Rail Station,None,U,529820.0,175710.0,2005-04-25,2006-09-18 18:24:34,1.0,rev,14.0,14,14.0,14.0,0.0,1.0,Monday
3,202407088097455,P97455,2024-07-08,Darwin,CRNG,1970-01-01 22:14:00,1970-01-01 22:14:30,1970-01-01 22:14:00,1970-01-01 22:14:00,1970-01-01 22:15:00,1970-01-01 22:15:00,2,False,False,9100CRNG,CRNG,CHG,Charing (Kent) Rail Station,None,U,595005.0,149132.0,2005-04-21,2006-09-18 18:24:34,1.0,rev,22.0,22,22.0,22.0,1.0,1.0,Monday
4,202407088090877,P90877,2024-07-08,TD,HOMRTON,1970-01-01 22:12:30,1970-01-01 22:13:00,1970-01-01 22:13:00,1970-01-01 22:13:00,1970-01-01 22:14:00,1970-01-01 22:14:00,2,False,False,9100HOMRTON,HOMRTON,HMN,Homerton Rail Station,None,U,535840.0,184940.0,2003-11-04,2010-05-14 17:37:23,2.0,rev,22.0,22,22.0,22.0,1.0,1.0,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79264,202407158097192,P97192,2024-07-15,Darwin,SYDNHMH,1970-01-01 20:41:30,1970-01-01 20:42:00,1970-01-01 20:42:00,1970-01-01 20:42:00,1970-01-01 20:42:00,1970-01-01 20:42:00,2,False,False,9100SYDNHMH,SYDNHMH,SYH,Sydenham Hill Rail Station,None,U,533540.0,172160.0,2005-04-25,2006-09-18 18:24:34,1.0,rev,20.0,20,20.0,20.0,0.0,0.0,Monday
79265,202407157111651,G11651,2024-07-15,Darwin,ASHWELC,1970-01-01 20:27:30,1970-01-01 20:28:00,1970-01-01 20:28:00,1970-01-01 20:28:00,NaT,1970-01-01 20:35:00,2,False,False,9100ASHWELC,ASHWELC,AWM,Ashwell & Morden Rail Station,None,U,529770.0,238615.0,2005-04-21,2006-09-18 18:24:34,1.0,rev,20.0,20,NaN,20.0,NaN,7.0,Monday
79266,202407158090093,P90093,2024-07-15,CIS,WNSTDPK,1970-01-01 21:14:30,1970-01-01 21:15:30,1970-01-01 21:15:00,1970-01-01 21:15:00,1970-01-01 21:15:00,1970-01-01 21:15:00,1,False,False,9100WNSTDPK,WNSTDPK,WNP,Wanstead Park Rail Station,None,U,540580.0,185590.0,2005-04-25,2006-09-18 18:24:34,1.0,rev,21.0,21,21.0,21.0,0.0,0.0,Monday
79267,202407157131940,G31940,2024-07-15,TD,TODMRDN,1970-01-01 20:36:00,1970-01-01 20:37:00,1970-01-01 20:36:00,1970-01-01 20:37:00,1970-01-01 20:37:00,1970-01-01 20:38:00,1,False,False,9100TODMRDN,TODMRDN,TOD,Todmorden Rail Station,None,U,393520.0,424160.0,2005-03-18,2006-09-18 18:24:34,1.0,rev,20.0,20,20.0,20.0,1.0,1.0,Monday


## Adjusting StationName Column for Better Readibility

With nulls in the StationName column, we filled those with their respective train platforms

In [24]:
# Fill null Station Names with its respective Train Platform
pandas_df['StationName'] = pandas_df['StationName'].fillna(pandas_df['train_platform'])

Each station's name read "Rail Station" at the end. Those were eliminated for redundency and readibility.

In [25]:
# Eliminate "Rail Station" for better readibility
pandas_df['StationName'] = pandas_df['StationName'].str.replace(' Rail Station', '')


## Creating Latitude & Longitude Columns

For further analysis with weather, the Easting & Northing columns were converted to obtain the proper weather data. By obtaining latitude and longitude data, we can analyze train delays at precise geographical locations, visualize patterns, and identify regions where trains might be delayed due to weather conditions. This allows for operational improvements, such as resource allocation and scheduling adjustments, to be planned and adjusted in anticipation of future weather impacts.

In [26]:
# Drop nulls before conversion
pandas_df = pandas_df.dropna(subset=['Easting', 'Northing'])

In [27]:
from pyproj import Proj, Transformer

# Define data from the National British Grid
# British National Grid (EPSG:27700)
bng = 'epsg:27700'
# WGS84 (EPSG:4326)
wgs84 = 'epsg:4326'

# Define the transformer for converting BNG to WGS84
transformer = Transformer.from_crs(bng, wgs84)

# Function to convert Easting & Northing to latitude & longitude
def convert_to_latlong(easting, northing):
    longitude, latitude = transformer.transform(easting, northing)
    return longitude, latitude

# Create Latitude & Longitude columns in the DataFrame
pandas_df[['Latitude', 'Longitude']] = pandas_df.apply(lambda row: convert_to_latlong(row['Easting'], row['Northing']),\
                                         axis=1,\
                                         result_type='expand')

/tmp/ipykernel_135/2293950844.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_df[['Latitude', 'Longitude']] = pandas_df.apply(lambda row: convert_to_latlong(row['Easting'], row['Northing']),\
/tmp/ipykernel_135/2293950844.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_df[['Latitude', 'Longitude']] = pandas_df.apply(lambda row: convert_to_latlong(row['Easting'], row['Northing']),\


In [29]:
pandas_df

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,is_delayed_arrival,is_delayed_departure,AtcoCode,TiplocCode,CrsCode,StationName,StationNameLang,GridType,Easting,Northing,CreationDateTime,ModificationDateTime,RevisionNumber,Modification,planned_arrival_hour,planned_dep_hour,actual_arrival_hour,actual_dep_hour,arrival_time_delayed_by(min),dep_time_delayed_by(min),day_of_week,Latitude,Longitude
0,202407086746668,C46668,2024-07-08,TD,DONC,1970-01-01 14:08:00,1970-01-01 14:10:00,1970-01-01 14:08:00,1970-01-01 14:10:00,NaT,1970-01-01 14:54:00,4,False,False,9100DONC,DONC,DON,Doncaster,None,U,457125.0,403175.0,2003-11-04 00:00:00,2010-12-20 15:37:54,2.0,rev,14.0,14,NaN,14.0,NaN,44.0,Monday,53.522148,-1.139845
2,202407088094162,P94162,2024-07-08,Trust,CLPHHS,1970-01-01 14:52:30,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:54:00,2,False,False,9100CLPHHS,CLPHHS,CLP,Clapham High Street,None,U,529820.0,175710.0,2005-04-25 00:00:00,2006-09-18 18:24:34,1.0,rev,14.0,14,14.0,14.0,0.0,1.0,Monday,51.465479,-0.132522
3,202407088097455,P97455,2024-07-08,Darwin,CRNG,1970-01-01 22:14:00,1970-01-01 22:14:30,1970-01-01 22:14:00,1970-01-01 22:14:00,1970-01-01 22:15:00,1970-01-01 22:15:00,2,False,False,9100CRNG,CRNG,CHG,Charing (Kent),None,U,595005.0,149132.0,2005-04-21 00:00:00,2006-09-18 18:24:34,1.0,rev,22.0,22,22.0,22.0,1.0,1.0,Monday,51.208098,0.790331
4,202407088090877,P90877,2024-07-08,TD,HOMRTON,1970-01-01 22:12:30,1970-01-01 22:13:00,1970-01-01 22:13:00,1970-01-01 22:13:00,1970-01-01 22:14:00,1970-01-01 22:14:00,2,False,False,9100HOMRTON,HOMRTON,HMN,Homerton,None,U,535840.0,184940.0,2003-11-04 00:00:00,2010-05-14 17:37:23,2.0,rev,22.0,22,22.0,22.0,1.0,1.0,Monday,51.547010,-0.042360
6,202407088094180,P94180,2024-07-08,TD,WAPPING,1970-01-01 16:44:00,1970-01-01 16:44:30,1970-01-01 16:44:00,1970-01-01 16:44:00,NaT,NaT,1,False,False,9100WAPPING,WAPPING,WPE,Wapping,None,U,535025.0,180175.0,2010-02-05 15:59:31,2010-02-05 15:59:31,0.0,new,16.0,16,NaN,NaN,NaN,NaN,Monday,51.504386,-0.055931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79264,202407158097192,P97192,2024-07-15,Darwin,SYDNHMH,1970-01-01 20:41:30,1970-01-01 20:42:00,1970-01-01 20:42:00,1970-01-01 20:42:00,1970-01-01 20:42:00,1970-01-01 20:42:00,2,False,False,9100SYDNHMH,SYDNHMH,SYH,Sydenham Hill,None,U,533540.0,172160.0,2005-04-25 00:00:00,2006-09-18 18:24:34,1.0,rev,20.0,20,20.0,20.0,0.0,0.0,Monday,51.432712,-0.080340
79265,202407157111651,G11651,2024-07-15,Darwin,ASHWELC,1970-01-01 20:27:30,1970-01-01 20:28:00,1970-01-01 20:28:00,1970-01-01 20:28:00,NaT,1970-01-01 20:35:00,2,False,False,9100ASHWELC,ASHWELC,AWM,Ashwell & Morden,None,U,529770.0,238615.0,2005-04-21 00:00:00,2006-09-18 18:24:34,1.0,rev,20.0,20,NaN,20.0,NaN,7.0,Monday,52.030770,-0.109789
79266,202407158090093,P90093,2024-07-15,CIS,WNSTDPK,1970-01-01 21:14:30,1970-01-01 21:15:30,1970-01-01 21:15:00,1970-01-01 21:15:00,1970-01-01 21:15:00,1970-01-01 21:15:00,1,False,False,9100WNSTDPK,WNSTDPK,WNP,Wanstead Park,None,U,540580.0,185590.0,2005-04-25 00:00:00,2006-09-18 18:24:34,1.0,rev,21.0,21,21.0,21.0,0.0,0.0,Monday,51.551691,0.026213
79267,202407157131940,G31940,2024-07-15,TD,TODMRDN,1970-01-01 20:36:00,1970-01-01 20:37:00,1970-01-01 20:36:00,1970-01-01 20:37:00,1970-01-01 20:37:00,1970-01-01 20:38:00,1,False,False,9100TODMRDN,TODMRDN,TOD,Todmorden,None,U,393520.0,424160.0,2005-03-18 00:00:00,2006-09-18 18:24:34,1.0,rev,20.0,20,20.0,20.0,1.0,1.0,Monday,53.713816,-2.099659


**Categorizing longitude & latitude columns into regions**

Using the calculated latitude and longitude values, we catoergorized them into respective geogrpahical regions. 

In [30]:
# Create regions: Northern, Southern, Eastern, Western, & Central
northern = 54.5  # Approx. above Yorkshire
southern = 51.0  # Approx. below London
eastern = -0.5   # East of the Prime Meridian
western = -3.0   # West of the Prime Meridian
# All else will return as Central

# Function to categorize regions
def categorize_region(latitude, longitude):
    if latitude >= northern:
        return 'Northern'
    elif latitude < southern:
        return 'Southern'
    elif longitude >= eastern:
        return 'Eastern'
    elif longitude < western:
        return 'Western'
    else:
        return 'Central'

# Create new column for regions
pandas_df['Region'] =pandas_df.apply(lambda row: categorize_region(row['Latitude'], row['Longitude']), axis=1)


/tmp/ipykernel_135/4283379652.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_df['Region'] =pandas_df.apply(lambda row: categorize_region(row['Latitude'], row['Longitude']), axis=1)


In [31]:
pandas_df.head(10)

,route_id,unique_id,service_start_date,update_origin,train_platform,working_time_arrival,working_time_departure,planned_time_arrival,planned_time_departure,actual_arrival_time,actual_departure_time,platform,is_delayed_arrival,is_delayed_departure,AtcoCode,TiplocCode,CrsCode,StationName,StationNameLang,GridType,Easting,Northing,CreationDateTime,ModificationDateTime,RevisionNumber,Modification,planned_arrival_hour,planned_dep_hour,actual_arrival_hour,actual_dep_hour,arrival_time_delayed_by(min),dep_time_delayed_by(min),day_of_week,Latitude,Longitude,Region
0,202407086746668,C46668,2024-07-08,TD,DONC,1970-01-01 14:08:00,1970-01-01 14:10:00,1970-01-01 14:08:00,1970-01-01 14:10:00,NaT,1970-01-01 14:54:00,4,False,False,9100DONC,DONC,DON,Doncaster,None,U,457125.0,403175.0,2003-11-04 00:00:00,2010-12-20 15:37:54,2.0,rev,14.0,14,NaN,14.0,NaN,44.0,Monday,53.522148,-1.139845,Central
2,202407088094162,P94162,2024-07-08,Trust,CLPHHS,1970-01-01 14:52:30,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:53:00,1970-01-01 14:54:00,2,False,False,9100CLPHHS,CLPHHS,CLP,Clapham High Street,None,U,529820.0,175710.0,2005-04-25 00:00:00,2006-09-18 18:24:34,1.0,rev,14.0,14,14.0,14.0,0.0,1.0,Monday,51.465479,-0.132522,Eastern
3,202407088097455,P97455,2024-07-08,Darwin,CRNG,1970-01-01 22:14:00,1970-01-01 22:14:30,1970-01-01 22:14:00,1970-01-01 22:14:00,1970-01-01 22:15:00,1970-01-01 22:15:00,2,False,False,9100CRNG,CRNG,CHG,Charing (Kent),None,U,595005.0,149132.0,2005-04-21 00:00:00,2006-09-18 18:24:34,1.0,rev,22.0,22,22.0,22.0,1.0,1.0,Monday,51.208098,0.790331,Eastern
4,202407088090877,P90877,2024-07-08,TD,HOMRTON,1970-01-01 22:12:30,1970-01-01 22:13:00,1970-01-01 22:13:00,1970-01-01 22:13:00,1970-01-01 22:14:00,1970-01-01 22:14:00,2,False,False,9100HOMRTON,HOMRTON,HMN,Homerton,None,U,535840.0,184940.0,2003-11-04 00:00:00,2010-05-14 17:37:23,2.0,rev,22.0,22,22.0,22.0,1.0,1.0,Monday,51.547010,-0.042360,Eastern
6,202407088094180,P94180,2024-07-08,TD,WAPPING,1970-01-01 16:44:00,1970-01-01 16:44:30,1970-01-01 16:44:00,1970-01-01 16:44:00,NaT,NaT,1,False,False,9100WAPPING,WAPPING,WPE,Wapping,None,U,535025.0,180175.0,2010-02-05 15:59:31,2010-02-05 15:59:31,0.0,new,16.0,16,NaN,NaN,NaN,NaN,Monday,51.504386,-0.055931,Eastern
7,202407088085975,P85975,2024-07-08,Darwin,SHRHMBS,1970-01-01 17:49:30,1970-01-01 17:50:30,1970-01-01 17:50:00,1970-01-01 17:50:00,1970-01-01 17:50:00,1970-01-01 17:50:00,1,False,False,9100SHRHMBS,SHRHMBS,SSE,Shoreham-by-Sea (Sussex),None,U,521800.0,105300.0,2003-11-04 00:00:00,2007-02-13 17:06:21,3.0,rev,17.0,17,17.0,17.0,0.0,0.0,Monday,50.834426,-0.271720,Southern
8,202407088085281,P85281,2024-07-08,TD,CLPHMJC,1970-01-01 22:37:00,1970-01-01 22:38:00,1970-01-01 22:37:00,1970-01-01 22:38:00,1970-01-01 22:55:00,1970-01-01 22:56:00,12,False,False,9100CLPHMJC,CLPHMJC,CLJ,Clapham Junction,None,U,527201.0,175500.0,2003-11-04 00:00:00,2007-11-14 16:23:42,2.0,rev,22.0,22,22.0,22.0,18.0,18.0,Monday,51.464186,-0.170279,Eastern
9,202407088082098,P82098,2024-07-08,TD,KRKH,1970-01-01 16:51:30,1970-01-01 16:52:00,1970-01-01 16:52:00,1970-01-01 16:52:00,1970-01-01 17:00:00,1970-01-01 17:00:00,1,True,True,9100KRKH,KRKH,KKH,Kirkhill,None,U,264300.0,659975.0,2003-11-04 00:00:00,2011-07-07 16:17:02,2.0,rev,16.0,16,17.0,17.0,8.0,8.0,Monday,55.813912,-4.167105,Northern
10,202407087136551,G36551,2024-07-08,Darwin,HBDNBDG,1970-01-01 22:37:00,1970-01-01 22:38:00,1970-01-01 22:37:00,1970-01-01 22:38:00,1970-01-01 22:44:00,1970-01-01 22:45:00,1,True,True,9100HBDNBDG,HBDNBDG,HBD,Hebden Bridge,None,U,399500.0,426800.0,2003-11-04 00:00:00,2006-09-18 18:24:34,1.0,rev,22.0,22,22.0,22.0,7.0,7.0,Monday,53.737586,-2.009059,Central
11,202407087131413,G31413,2024-07-08,Darwin,ASHBRYS,1970-01-01 16:44:00,1970-01-01 16:44:30,1970-01-01 16:44:00,1970-01-01 16:44:00,NaT,1970-01-01 16:45:00,2,False,False,9100ASHBRYS,ASHBRYS,ABY,Ashburys,None,U,387191.0,397229.0,2003-11-04 00:00:00,2009-01-05 10:01:28,2.0,rev,16.0,16,NaN,16.0,NaN,1.0,Monday,53.471636,-2.1944

## Describe/ Explore Data

After cleaning the data, the following Data Dictionary defines what will be analyzed:

**Data Dictionary**
- route_id: The unique identifier for each train route.
- unique_id: The unique identifier for each train
- planned_time_arrival: The officially planned time for the train to arrive at a station. This could be the public-facing scheduled time.
- planned_time_departure: The officially planned time for the train to depart from a station. This could be the public-facing scheduled time.
- platform: The platform number at the station where the train arrives and departs.
- actual_arrival_time: The actual time when the train arrived at a station.
- actual_departure_time: The actual time when the train departed from a station.
- is_delayed_arrival: A boolean flag indicating whether the train arrived late
- is_delayed_departure: A boolean flag indicating whether the train departed late from a station.
- arrival_time_delayed_by(min): Time in mintues the train is delayed arriving.
- dep_time_delayed_by(min): Time in mintues the train is delayed departing.
- day_of_week: The day of the week a train is running.
- latitude: Latitude train coordinates.
- longitude: Longitute train coordinates.
- reagion: Region of the UK the latitude & longitude cooridates correlate to.

In [ ]:
# Create new delayed DataFrame 

delayed_df = pandas_df[['route_id', 'unique_id', 'train_platform', 'StationName', 'day_of_week', 'planned_arrival_hour', 'is_delayed_arrival', 'arrival_time_delayed_by(min)', 'is_delayed_departure', 'planned_dep_hour', 'dep_time_delayed_by(min)', 'actual_arrival_time', 'actual_arrival_hour', 'actual_dep_hour', 'actual_departure_time', 'Longitude','Latitude','Region']]
delayed_df = delayed_df[delayed_df['is_delayed_departure'] == True]
delayed_df.head(30)

## Train Cancellations

In [ ]:
# Calculate the number of cancellations
cancellations = pandas_df['actual_dep_hour'].isnull().sum()
print(f"Number of cancellations: {cancellations}")

# Calculate the total number of planned departures
total_planned = len(pandas_df)
print(f"Total number of planned departures: {total_planned}")

# Calculate the cancellation rate
cancellation_rate = (cancellations / total_planned) * 100
print(f"Cancellation rate: {cancellation_rate:.2f}%")

With a relativly low number of train cancellations, this is not a concerning factor, but one to consider when analyzing delays. Rows with cancelled trains will be dropped.

In [ ]:
pandas_df

In [ ]:
# Drop rows with null depature hours
#pandas_df = pandas_df.dropna(subset=['actual_dep_hour'])

## Analysis 1
## During Which Hour of the Day and Day of the Week Are Trains Most Frequently Delayed?
Here, we visualize the comparion between the frequency of actual vs planned train departures.

In [ ]:
bins = [0, 5, 10, 15, 20, 25]
labels = ['Early Morning (12am)', 'Morning (5am)', 'Midday(10am)', 'Evening (3pm)', 'Night (8pm)', ' ']

plt.figure(figsize=(14,7))

# Planned departure time distribution
plt.subplot(1, 2, 1)
plt.hist(delayed_df['planned_dep_hour'], bins=bins, color='purple')
plt.title('Distribution of Planned Train Departure Hour')
plt.xlabel('Departure Hour')
plt.ylabel('Train Frequency')
plt.xticks(bins, labels, rotation=45)
plt.grid()

# Actual departure time distribution
plt.subplot(1, 2, 2)
plt.hist(delayed_df['actual_dep_hour'], bins=bins, color='blue')
plt.title('Distribution of Actual Train Departure Hour')
plt.xlabel('Departure Hour')
plt.ylabel('Train Frequency')
plt.xticks(bins, labels, rotation=45)

plt.grid()
plt.tight_layout()
plt.show()

Train departures are most frequent during the evening (3pm-7pm) and night (8pm-11pm) periods. This increased volume of departures during these times could potentially contribute to a higher likelihood of delays. There is a noticeable reduction in the number of actual departures compared to the planned departures, particularly in the "Evening (3pm)" and "Night (8pm)" categories.

In [ ]:
bins = [0, 5, 10, 15, 20, 25]
labels = ['Early Morning (12am)', 'Morning (5am)', 'Midday(10am)', 'Evening (3pm)', 'Night (8pm)', ' ']

plt.figure(figsize=(14,7))

# Planned arrival time distribution
plt.subplot(1, 2, 1)
plt.hist(pandas_df['planned_arrival_hour'], bins=bins, color='purple')
plt.title('Distribution of Planned Train Departure Hour')
plt.xlabel('Departure Hour')
plt.ylabel('Train Frequency')
plt.xticks(bins, labels, rotation=45)
plt.grid()

# Actual arrival time distribution
plt.subplot(1, 2, 2)
plt.hist(pandas_df['actual_arrival_hour'], bins=bins, color='blue')
plt.title('Distribution of Actual Train Departure Hour')
plt.xlabel('Departure Hour')
plt.ylabel('Train Frequency')
plt.xticks(bins, labels, rotation=45)


plt.grid()
plt.tight_layout()
plt.show()

## Delays by Day of Week

Categorizing delays by the day of the week allows us to determine if delays are more frequent on certain days or if specific days consistently experience higher delays. Such inforamtion can be useful for future scheudling purposes such as adding more buffer time to mitigate dealys or increase resources, such as number of tains running and availability of staff & maintenence workers.

**Creating a Weekday Data Frame to plot**

In [ ]:
import pandas as pd

# Grouping delayed data by day of the week and calculating the mean for numeric columns
weekday_df = delayed_df.groupby('day_of_week').mean(numeric_only=True).reset_index()

# Defining the order of the days of the week
days_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Creating a categorical column to ensure the correct order of the days
weekday_df['day'] = pd.Categorical(weekday_df['day_of_week'], categories=days_order, ordered=True)

# Sort DataFrame by 'day' column
weekday_df = weekday_df.sort_values('day') 
weekday_df

**Day of Week Distribution**

The following charts display a distribution of data per day, noting no data collected on Saturday. 

In [ ]:
plt.figure(figsize=(10, 6))
day_dist = pandas_df['day_of_week'].value_counts()
sns.barplot(x=day_dist.index, y=day_dist.values, color='purple')
plt.title('Day of Week Distribution')
plt.xlabel('Day')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
delay_day_dist = delayed_df['day_of_week'].value_counts()
sns.barplot(x=delay_day_dist.index, y=delay_day_dist.values, color='purple')
plt.title('Delays by Day of Week')
plt.xlabel('Day')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

While the most number of trains run Wednesday & Thursday, the most delays happen Friday and Monday.

**Hour & day of week distribution**

The following heatmap visualizes the frequency of delayed train departures categorized by the time of day and day of the week. 

In [ ]:
# Define bins and labels for departure hours
bins = [0, 5, 10, 15, 20, 25]
labels = ['Early Morning (12am)', 'Morning (5am)', 'Midday (10am)', 'Afternoon (3pm)', 'Evening (8pm)']

# Pivot table to count frequencies of dep_hour_category by day_of_week
heatmap_data = delayed_df.pivot_table(index='day_of_week', columns='actual_dep_hour', aggfunc='size', fill_value=0)

plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data.loc[days], cmap='coolwarm', annot=True, fmt='d', cbar=True)
plt.title('Frequency of Departure Hour by Day of Week')
plt.xlabel('Departure Hour')
plt.ylabel('Day of Week')
plt.xticks(rotation=45)

plt.show()


## Analysis 1: Conclusion:

**Busiest days:** 
- Wednesday & Friday have the highest frequency of train departures.
- Monday shows a distinct pattern with a high number of departures in the afternoon (3pm-8pm) and evening (8pm-12am) but almost none in the morning (5am-10am).
- Friday stands out with the highest number of departures during the afternoon (3pm-8pm) and significant activity in midday (10am-3pm).

**Least busy days:** The weekends run the least amount of departures.

**Peak hours:**
- The highest frequency of departures occurs during the afternoon (3pm-8pm) on weekdays, with Wednesday, Thursday, and Friday showing particularly high numbers.
- Midday (10am-3pm) and evening (8pm-12am) also see substantial departure activity, especially on Wednesday and Friday.
  
**Slow hours:**
- Early morning (12am-5am) shows the least train departure activity across all days, with only a few departures observed.
- Similarly, morning (5am-10am) on Monday and Tuesday shows relatively low departure frequencies.

These patterns indicate that train departure schedules are highly concentrated on weekdays during afternoon and evening hours, which could be reflective of peak travel times. OVer time, as more data is collected, it can be analyzed for more accruate delayed arrival and departures. 

In [ ]:
# create new df with route_id, unique_id, StationName, isDelayed
# delayed_df = pandas_df[pandas_df['is_delayed_departure'] == True]
# delayed_df.head()

## Analysis 2
Do the most popular stations also have the most delays?

**Top 10 most Frequently Visited Stations**

In [ ]:
top_10_stations = pandas_df['StationName'].value_counts().head(20)
plt.figure(figsize=(10, 6))
sns.barplot(x=top_10_stations.index, y=top_10_stations.values)
plt.title('Top 10 Most Frequently Visited Stations')
plt.xlabel('Station Name')
plt.ylabel('Count of Visits')
plt.xticks(rotation=90)
plt.show()

Clapham Junction is the most frequently visited station. As a major transport hub, it houses connections to popular rail lines including South Western Railway, Southern, and London Overground, as well as access to major destinations such as Gatwick Airport, London Waterloo, and London Victoria. Servicing densely populated residential areas in South West London, and close proximity to business and commercial districts contribute to the increase in passenger demand.

**Top 10 stations with most delays**

In [ ]:
result = pandas_df.groupby('StationName')['dep_time_delayed_by(min)'].sum().reset_index()
result = result.sort_values(by='dep_time_delayed_by(min)', ascending=False)
result.head(10)

**Top 10 stations with count of delayed departures**

In [ ]:
# Find top 10 stations with most delays
result = pandas_df[pandas_df['is_delayed_departure'] == True].groupby('StationName')['is_delayed_departure'].count().reset_index()
result = result.sort_values(by='is_delayed_departure', ascending=False)
result.head(10)

**Top 10 Stations delayed by avergae minutes**

In [ ]:

avg_delays = pandas_df.groupby('StationName')[['arrival_time_delayed_by', 'dep_time_delayed_by']].mean().reset_index()
avg_delays = avg_delays.sort_values(by=['arrival_time_delayed_by', 'dep_time_delayed_by'], ascending=False)
avg_delays.head(10)

## Analysis 3: Correlations between Stations and their Delays

In [ ]:
The following correlation analyses will help us understand the relationship betw

**Delay Correlation Analysis**

In [ ]:
# Analyze correlations between features and delay columns
correlation_matrix = pandas_df.corr(numeric_only=True)
# Display the correlations with delay columns
delay_correlations = correlation_matrix[['arrival_time_delayed_by', 'dep_time_delayed_by', ]].sort_values(by='arrival_time_delayed_by', ascending=False)
# Display the top correlations
delay_correlations.head()

While arrival and delay have the strongest correlation, that is anticipated, as if a trian was arriving late, it would liekly be departing late as well. This correlation is not 

**Comprehensive Correlation Analysis**

In [ ]:
# Calculate the correlation matrix for the dataset
correlation_matrix = pandas_df.corr(numeric_only=True)

# Sort the correlation values to find the highest correlations
correlation_sorted = correlation_matrix.abs().unstack().sort_values(ascending=False)

# Remove self-correlations (correlation of a feature with itself)
correlation_sorted = correlation_sorted[correlation_sorted < 1]

# Display the highest correlations
correlation_sorted.head(10)

**Correlation Matrix**

In [ ]:
# Visual: Correlation Heat Map:
corr_map = pandas_df[['planned_arrival_hour', 'planned_dep_hour','actual_arrival_hour', 'actual_dep_hour', 'Latitude', 'Longtitude']]
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()

Delayed arrived and delayed departures are highly correlated, while geospatial features such as Easting and Northing have low correlation with delays. Such geospatial features will not be useful to analyze further

## Logistic Regression
Created a LR Model to predict if the train departure will be delayed

In [ ]:
# logistic regression:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Select relevant columns and drop null rows
vars = pandas_df[['train_platform', 'planned_arrival_hour', 'planned_dep_hour', 'is_delayed_departure']].dropna()

# Convert booleans to floats
vars['is_delayed_departure'] = vars['is_delayed_departure'].astype('float')
vars['planned_dep_hour'] = vars['planned_dep_hour'].astype('float')

# One-hot encoding for the categorical 'platform' column
vars = pd.get_dummies(vars, columns=['train_platform'], dtype='float')

# X = all columns except 'is_delayed_departure'
X = vars.drop('is_delayed_departure', axis=1)
y = vars['is_delayed_departure']

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Train Model
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

In [ ]:
# Evaluate Model
y_pred = log_reg.predict(X_test)

# Define report, confusion matrix, and accuracy
report = classification_report(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

# Display results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)
print("Confusion Matrix:\n", cm)

he model correctly predicted the negative class:
- True Negatives = 12,704
- False Positives = 19
The model correctly predicted the positive class:
- True Positives = 40
- False Negatives = 924

In [ ]:
# Visual: confusion matrix

import matplotlib.pyplot as plt
import seaborn as sns
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['No Delay', 'Delay'], yticklabels=['No Delay', 'Delay'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Overall Review

The scheduling of more or fewer trains at certain times is a complex decision influenced by passenger demand, operational efficiency, maintenance needs, geographical and demographic factors, and strategic business considerations. By aligning train schedules with these factors, train operators aim to provide efficient and reliable service while managing costs and maximizing customer satisfaction.

## Export DataFrame to PowerBi for Further Visualizations

In [ ]:
pandas_df.head()

In [32]:
pandas_df.to_csv('rail_data_eda.csv', index=False)